# Streaming

Sensors, IoT devices, social networks, and online transactions all generate data that needs to be monitored constantly and acted upon quickly. As a result, the need for large-scale, real-time stream processing is more evident than ever before. This tutorial module introduces Structured Streaming, the main model for handling streaming datasets in Apache Spark. *In Structured Streaming, a data stream is treated as a table that is being continuously appended*. This leads to a stream processing model that is very similar to a batch processing model. You express your streaming computation as a standard batch-like query as on a static table, but Spark runs it as an incremental query on the unbounded input table.


## Load data

Databricks has sample event data as files in`/databricks-datasets/structured-streaming/events/` to use to build a Structured Streaming application. Let's take a look at the contents of this directory.

In [0]:
%fs ls /databricks-datasets/structured-streaming/events/

path,name,size,modificationTime
dbfs:/databricks-datasets/structured-streaming/events/file-0.json,file-0.json,72530,1596690605000
dbfs:/databricks-datasets/structured-streaming/events/file-1.json,file-1.json,72961,1596690605000
dbfs:/databricks-datasets/structured-streaming/events/file-10.json,file-10.json,73025,1596690605000
dbfs:/databricks-datasets/structured-streaming/events/file-11.json,file-11.json,72999,1596690605000
dbfs:/databricks-datasets/structured-streaming/events/file-12.json,file-12.json,72987,1596690605000
dbfs:/databricks-datasets/structured-streaming/events/file-13.json,file-13.json,73006,1596690605000
dbfs:/databricks-datasets/structured-streaming/events/file-14.json,file-14.json,73003,1596690605000
dbfs:/databricks-datasets/structured-streaming/events/file-15.json,file-15.json,73007,1596690605000
dbfs:/databricks-datasets/structured-streaming/events/file-16.json,file-16.json,72978,1596690605000
dbfs:/databricks-datasets/structured-streaming/events/file-17.json,file-17.json,73008,1596690605000


Each line in the file contains a JSON record with two fields: `time` and `action`.

```
{"time":1469501675,"action":"Open"}
{"time":1469501678,"action":"Close"}{"time":1469501680,"action":"Open"}{"time":1469501685,"action":"Open"}{"time":1469501686,"action":"Open"}{"time":1469501689,"action":"Open"}{"time":1469501691,"action":"Open"}{"time":1469501694,"action":"Open"}{"time":1469501696,"action":"Close"}{"time":1469501702,"action":"Open"}{"time":1469501703,"action":"Open"}{"time":1469501704,"action":"Open"}
```

## Initialize the stream

Since the sample data is just a static set of files, you can emulate a stream from them by reading one file at a time, in the chronological order in which they were created:

```python
streamingInputDF = (
  spark.readStream\
    .schema(jsonSchema)               # Set the schema of the JSON data
    .option("maxFilesPerTrigger", 1)  # Treat a sequence of files as a stream by picking one file at a time
    .json(inputPath)
)
```

In [0]:
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, TimestampType
from pyspark.sql.functions import window

In [0]:
inputPath = "/databricks-datasets/structured-streaming/events/"

# Define the schema to speed up processing
jsonSchema = StructType([ StructField("time", TimestampType(), True), StructField("action", StringType(), True) ])

streamingInputDF = (
  spark.readStream\
    .schema(jsonSchema)\
    .option("maxFilesPerTrigger", 1)\
    .json(inputPath)
)

streamingCountsDF = (
  streamingInputDF
    .groupBy(
      streamingInputDF.action,
      window(streamingInputDF.time, "1 hour"))
    .count()
)

You start a streaming computation by defining a sink and starting it. In our case, to query the counts interactively, set the complete set of 1 hour counts to be in an in-memory table. The command below essentially simulates a time series generator in real time, using the files in the directory.

In [0]:
query = (
  streamingCountsDF
    .writeStream
    .format("memory")        # memory = store in-memory table (for testing only)
    .queryName("counts")     # counts = name of the in-memory table
    .outputMode("complete")  # complete = all the counts should be in the table
    .start()
)

---------------------------------------------------------------------------
AnalysisException                         Traceback (most recent call last)
File <command-5892011683551895>, line 7
      1 query = (
      2   streamingCountsDF
      3     .writeStream
      4     .format("memory")        # memory = store in-memory table (for testing only)
      5     .queryName("counts")     # counts = name of the in-memory table
      6     .outputMode("complete")  # complete = all the counts should be in the table
----> 7     .start()
      8 )

File /databricks/python/lib/python3.11/site-packages/pyspark/sql/connect/streaming/readwriter.py:648, in DataStreamWriter.start(self, path, format, outputMode, partitionBy, queryName, **options)
    639 def start(
    640     self,
    641     path: Optional[str] = None,
   (...)
    646     **options: "OptionalPrimitiveType",
    647 ) -> StreamingQuery:
--> 648     return self._start_internal(
    649         path=path,
    650         tableName=

`query` is a handle to the streaming query named `counts` that is running in the background. This query continuously picks up files and updates the windowed counts. The command window reports the status of the stream.

Sadly, we cannot proceed in databricks because we cannot start a continuously updating streaming process. But if we could, we could periodically query the counts aggregation:

```sql
%sql select action, date_format(window.end, "MMM-dd HH:mm") as time, count from counts order by time, action
```

The query would change every time you execute it to reflect the action count based on the input stream of data.